In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import (
    NoSuchElementException,
    NoAlertPresentException,
    NoSuchWindowException,
    TimeoutException,
    UnexpectedAlertPresentException,
)
import time
import json
import traceback
import datetime
import logging

# 로깅 설정
logging.basicConfig(
    filename='ticketing_script.log',
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    encoding='utf-8'  # 인코딩을 utf-8로 설정
)

# ChromeDriver 서비스 설정
driver_path = './chromedriver.exe'  # ChromeDriver 경로를 지정하세요
service = Service(executable_path=driver_path)

# Chrome 실행 옵션 설정
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화 탐지 방지 옵션
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option('useAutomationExtension', False)
driver = webdriver.Chrome(service=service, options=chrome_options)

# 최대 시도 횟수 및 모니터링 간격 설정
max_attempts = 10000
monitoring_interval = 0.2  # 초

# 1. Yes24 티켓 페이지로 이동 (쿠키 추가할 도메인으로 먼저 이동)
driver.get('https://ticket.yes24.com')
logging.info("Yes24 티켓 페이지로 이동했습니다.")

# 2. 저장된 쿠키 불러오기 (현재 도메인에 쿠키 추가)
try:
    with open('cookies.json', 'r') as cookies_file:
        cookies = json.load(cookies_file)
        for cookie in cookies:
            if 'sameSite' in cookie:
                del cookie['sameSite']
            driver.add_cookie(cookie)
    logging.info("쿠키 로드 성공")
except Exception as e:
    logging.error(f"쿠키 로드 실패: {e}")

# 3. 쿠키 추가 후 페이지 새로고침 (로그인 유지 확인)
driver.refresh()
logging.info("페이지 새로고침 완료")

# 4. 예매하기 버튼이 있는 페이지로 이동
driver.get('http://ticket.yes24.com/Special/51057')
logging.info("예매하기 페이지로 이동했습니다.")

# 5. 예매하기 버튼 클릭
try:
    book_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="mainForm"]/div[9]/div/div[4]/a[4]'))  # 예매하기 버튼 XPath
    )
    book_button.click()  # 예매하기 버튼 클릭
    logging.info("예매하기 버튼 클릭 완료")
except Exception as e:
    logging.error(f"예매하기 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 팝업 처리 함수
def handle_alert():
    try:
        alert = driver.switch_to.alert  # 팝업창이 있을 경우 처리
        logging.info(f"팝업 텍스트: {alert.text}")
        alert.accept()  # 팝업 닫기
    except NoAlertPresentException:
        pass  # 팝업이 없을 경우 아무 작업 안 함

# 팝업 처리 (필요할 때마다 호출)
handle_alert()

# 현재 열린 창의 핸들 저장
main_window = driver.current_window_handle

# 예매하기 버튼을 클릭한 후 새 창(또는 팝업창)이 열리는지 확인하고 제어를 전환
try:
    WebDriverWait(driver, 10).until(
        EC.number_of_windows_to_be(2)  # 새 창이 열릴 때 창의 수가 2개가 되길 기다림
    )
    all_windows = driver.window_handles
    for window in all_windows:
        if window != main_window:  # 새로운 창으로 제어를 넘김
            driver.switch_to.window(window)
            logging.info("새 창으로 제어를 전환했습니다.")
            break
except Exception as e:
    logging.error(f"새 창 제어 오류: {e}")
    driver.quit()
    exit()

# 날짜 선택
try:
    date_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, "2024-10-20"))  # 날짜 선택 (ID 수정 필요)
    )
    date_button.click()
    logging.info("날짜 선택 완료")
except Exception as e:
    logging.error(f"날짜 선택 오류: {e}")
    driver.quit()
    exit()

# 회차 선택 (1회차 선택)
try:
    time_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="ulTime"]/li[1]'))  # 회차 선택 (XPath 사용)
    )
    time_button.click()
    logging.info("회차 선택 완료")
except Exception as e:
    logging.error(f"회차 선택 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 버튼 클릭
try:
    seat_button = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.ID, 'btnSeatSelect'))  # 좌석 선택 버튼 (ID 사용)
    )
    seat_button.click()
    logging.info("좌석 선택 버튼 클릭 완료")
except Exception as e:
    logging.error(f"좌석 선택 버튼 클릭 오류: {e}")
    driver.quit()
    exit()

# 좌석 선택 및 검증 함수
def select_seat_by_id(seat_id):
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()

        # 좌석 선택 프레임으로 전환
        iframe_seat = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.NAME, 'ifrmSeatFrame'))
        )
        driver.switch_to.frame(iframe_seat)
        logging.info("좌석 선택 프레임으로 전환했습니다.")

        # 해당 좌석 요소 찾기
        seat = driver.find_element(By.ID, seat_id)
        logging.info(f"좌석 {seat_id}를 찾았습니다.")

        # 좌석 클릭을 JavaScript로 실행
        driver.execute_script("arguments[0].click();", seat)
        logging.info(f"좌석 {seat_id}를 클릭했습니다.")

        # 좌석 선택 상태 확인을 위해 약간의 대기 시간 추가
        time.sleep(0.5)  # 필요에 따라 시간 조절

        # JavaScript를 사용하여 좌석 상태 확인
        seat_selected = driver.execute_script(
            "return arguments[0].classList.contains('selected') || arguments[0].classList.contains('selected-seat');",
            seat
        )
        if seat_selected:
            logging.info(f"좌석 {seat_id}가 성공적으로 선택되었습니다.")
            return True
        else:
            logging.info(f"좌석 {seat_id} 선택 실패.")
            return False

        # 좌석선택완료 버튼 클릭
        confirm_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@href='javascript:ChoiceEnd();']"))
        )
        confirm_button.click()
        logging.info("좌석선택완료 버튼을 클릭했습니다.")

        return True  # 좌석 선택 및 확인 버튼 클릭 성공

    except NoSuchElementException:
        logging.error(f"좌석 {seat_id} 요소를 찾을 수 없습니다.")
        return False
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            logging.info(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        logging.error(f"좌석 선택에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 모니터링 및 시도 함수
def monitor_and_select_seat(seat_id):
    attempts = 0
    max_attempts = 10000
    monitoring_interval = 0.5  # 초

    while attempts < max_attempts:
        try:
            attempts += 1
            logging.info(f"Attempt {attempts} out of {max_attempts}")

            # 특정 횟수마다 진행 상황 메시지 출력
            if attempts % 100 == 0:
                current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                logging.info(f"[{current_time}] 스크립트가 정상적으로 실행 중입니다. Selenium이 잘 작동하고 있습니다.")

            # 좌석 선택 시도
            success = select_seat_by_id(seat_id)
            if success:
                logging.info("좌석 선택에 성공했습니다.")
                return True
            else:
                logging.info("좌석 선택에 실패했습니다. 다시 시도합니다.")

        except Exception as e:
            logging.error(f"모니터링 중 예기치 않은 오류 발생: {e}")
            traceback.print_exc()

        # 대기 후 다시 시도
        time.sleep(monitoring_interval)

    logging.info("최대 시도 횟수를 초과했습니다. 좌석 선택에 실패했습니다.")
    
    return False

# '다음단계' 버튼 클릭 함수
def click_next_step():
    try:
        # 기본 컨텐츠로 전환
        driver.switch_to.default_content()
        logging.info("Attempting to click '다음단계' 버튼")

        # '다음단계' 버튼이 로드될 때까지 기다림
        next_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//a[@onclick='fdc_PromotionEnd();']/img[@alt='다음단계']"))
        )
        # 부모 <a> 태그 클릭
        parent_element = next_button.find_element(By.XPATH, "..")
        parent_element.click()
        logging.info("'다음단계' 버튼을 클릭했습니다.")
        return True
    except UnexpectedAlertPresentException:
        try:
            alert = driver.switch_to.alert
            alert_text = alert.text
            logging.info(f"알림 창이 나타났습니다: {alert_text}")
            alert.accept()
        except NoAlertPresentException:
            pass
        return False
    except Exception as e:
        logging.error(f"'다음단계' 버튼 클릭에 실패했습니다: {e}")
        traceback.print_exc()
        return False

# 좌석 선택 시도
seat_id = 't4100023'  # 선택하려는 좌석 ID를 입력하세요
if monitor_and_select_seat(seat_id):
    logging.info("좌석 선택에 성공했습니다.")
    # '다음단계' 버튼 클릭
    if click_next_step():
        logging.info("다음 단계로 이동했습니다.")
    else:
        logging.info("다음 단계로 이동하지 못했습니다.")
else:
    logging.info("좌석 선택에 실패했습니다. 모니터링을 종료합니다.")

# 브라우저 유지 또는 종료
# 필요에 따라 주석을 해제하세요
# driver.quit()


Traceback (most recent call last):
  File "C:\Users\igotd\AppData\Local\Temp\ipykernel_3988\2299830254.py", line 171, in select_seat_by_id
    seat_selected = driver.execute_script(
                    ^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 414, in execute_script
    return self.execute(command, {"script": script, "args": converted_args})["value"]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Python312\Lib\site-packages\selenium\webdriver\remote\webdriver.py", line 354, in execute
    self.error_handler.check_response(response)
  File "c:\Python312\Lib\site-packages\selenium\webdriver\remote\errorhandler.py", line 229, in check_response
    raise exception_class(message, screen, stacktrace)
selenium.common.exceptions.StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=129.0.6668.101); For docume